In [1]:
# Import dependencies
import requests
from config import spotify_token
import pandas as pd
import numpy as np
import random

In [2]:
query_url = 'https://accounts.spotify.com/api/token'
results = requests.get(query_url, headers={
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {spotify_token}',
    'country': 'US'})

In [3]:
base_url = 'https://api.spotify.com/v1/'

In [4]:
# Create a list of all the 50 category ids
query_url = base_url + f'browse/categories?limit=50'
results = requests.get(query_url, headers={
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {spotify_token}',
    'country': 'US'}).json()

categories_list = []
for id, category in enumerate(results['categories']['items']):
    categories_list.append(category['id'])
categories_list

['toplists',
 '2020',
 'at_home',
 'pop',
 'mood',
 'decades',
 'hiphop',
 'in_the_car',
 'gaming',
 'wellness',
 'workout',
 'chill',
 'focus',
 'sleep',
 'party',
 'metal',
 'rock',
 'edm_dance',
 'dinner',
 'jazz',
 'rnb',
 'instrumental',
 'romance',
 'soul',
 'indie_alt',
 'classical',
 'family',
 'latin',
 'caribbean',
 'blues',
 'funk',
 'punk',
 'country',
 'roots',
 'desi',
 'arab',
 'afro',
 'travel',
 'kpop']

In [5]:
# Create a dictionary of categories and playlists
category_playlist_ids = {}
for cat in categories_list:
    query_url = base_url + f'browse/categories/{cat}/playlists?limit=5'
    playlists = []
    results = requests.get(query_url, headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {spotify_token}',
        'country': 'US'}).json()
    try: 
        for id, playlist in enumerate(results['playlists']['items']):
            playlists.append(playlist['id'])
        if len(playlists) > 0:
            category_playlist_ids[cat] = playlists
    except:
        print(f'No playlists for category: {cat}')
len(category_playlist_ids)

38

In [6]:
# Create a DataFrame of song names, artists, categories, and genres
song_artist_category_df = pd.DataFrame(columns=['song', 'song_id', 'artist', 'artist_id', 'category'])
song_artist_category_df
index = 0
for category in category_playlist_ids:
    songs_list = []
    # Determine how many songs to get from each playlist
    playlists = category_playlist_ids[category]
    len_limit = {1: 25, 2:10, 3: 8, 4: 6, 5: 5}
    length = len(category_playlist_ids[category])
    if length == 0:
        print(category)
    limit = len_limit[length]
    for playlist_id in playlists:
        # Get songs from each playlist
        query_url = base_url + f'playlists/{playlist_id}/tracks?limit={limit}'
        results = requests.get(query_url, headers={
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {spotify_token}',
            'country': 'US'}).json()
        try:
            # Get the song, artist, and genres for each song
            for id, item in enumerate(results['items']):
                song = item['track']['name']
                song_id = item['track']['id']
                artist = item['track']['artists'][0]['name']
                artist_id = item['track']['artists'][0]['id']
                song_artist_category_df.loc[index] = [song, song_id, artist, artist_id, category]
                index+=1
        except:
            print(f'No results for playlist {playlist_id}')
song_artist_category_df

No results for playlist 37i9dQZF1DX0XUsuxWHRQd
No results for playlist 37i9dQZF1DX0XUsuxWHRQd
No results for playlist 37i9dQZF1DX2RxBh64BHjQ


,song,song_id,artist,artist_id,category
0,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists
1,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists
2,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists
3,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists
...,...,...,...,...,...
921,Given-Taken,69WpV0U7OMNFGyq8I63dcC,ENHYPEN,5t5FqBwTcgKTaWmfEbwQY9,kpop
922,Christmassy!,1KMc5CXwEkjklwSqPMQjB1,THE BOYZ,0CmvFWTX9zmMNCUi6fHtAx,kpop
923,Better,2k44rKotfi2k55hwwiCImN,BoA,4muJrGMndyYWqZtfk8OWy4,kpop
924,Panorama,0CnpSTdL9l5vQM4YnzXtyo,IZ*ONE,5r1tUTxVSgvBHnoDuDODPH,kpop


In [7]:
# Create a dictionary of genres for each artist
artist_genres = {}
count = 0
for artist_id in song_artist_category_df['artist_id'].unique():
    query_url = base_url + f'artists/{artist_id}'
    artist = requests.get(query_url, headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {spotify_token}',
        'country': 'US'}).json()
    genres = artist['genres']
    genres = list(set([genre.split()[0] for genre in genres]))
    artist_genres[artist_id] = genres
    if genres == []:
        artist_name = artist['name']
        #print(f'No genres found for {artist_name}')
        count+=1
print(f'No genres found for {count} artists.')
len(artist_genres)

No genres found for 109 artists.


761

In [8]:
# Add a column of artist genres
genre_column = []
for artist_id in song_artist_category_df['artist_id']:
    genre_column.append(artist_genres[artist_id])
song_artist_category_df['genres'] = genre_column
song_artist_category_df

,song,song_id,artist,artist_id,category,genres
0,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[post-teen, dance, pop]"
1,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[dance, uk, pop]"
2,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[electropop, pop]"
3,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[dance, pop, canadian, post-teen, viral]"
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[queer, lgbtq+, country, pop]"
...,...,...,...,...,...,...
921,Given-Taken,69WpV0U7OMNFGyq8I63dcC,ENHYPEN,5t5FqBwTcgKTaWmfEbwQY9,kpop,[]
922,Christmassy!,1KMc5CXwEkjklwSqPMQjB1,THE BOYZ,0CmvFWTX9zmMNCUi6fHtAx,kpop,[k-pop]
923,Better,2k44rKotfi2k55hwwiCImN,BoA,4muJrGMndyYWqZtfk8OWy4,kpop,"[k-pop, j-idol]"
924,Panorama,0CnpSTdL9l5vQM4YnzXtyo,IZ*ONE,5r1tUTxVSgvBHnoDuDODPH,kpop,[k-pop]


In [9]:
# Create a list of all genres
genre_list = []
for artist_genres in genre_column:
    genre_list.extend(artist_genres)
genre_list = list(set(genre_list))
len(genre_list)

392

In [10]:
# Find most popular genres
popular_genres = dict.fromkeys(genre_list, 0)
for artist_genres in genre_column:
    for genre in artist_genres:
        popular_genres[genre]+=1
popular_genres = dict(sorted(popular_genres.items(), key=lambda kv: kv[1], reverse=True))
popular_genres

{'pop': 201,
 'dance': 105,
 'modern': 85,
 'rock': 75,
 'new': 64,
 'alternative': 63,
 'indie': 63,
 'classic': 59,
 'post-teen': 48,
 'soul': 47,
 'rap': 42,
 'funk': 41,
 'trap': 41,
 'contemporary': 41,
 'motown': 37,
 'tropical': 36,
 'hip': 35,
 'canadian': 32,
 'latin': 32,
 'quiet': 32,
 'country': 29,
 'jazz': 28,
 'background': 28,
 'uk': 27,
 'album': 26,
 'urban': 25,
 'mellow': 24,
 'k-pop': 22,
 'r&b': 22,
 'hard': 21,
 'stomp': 21,
 'permanent': 21,
 'melodic': 21,
 'soft': 21,
 'desi': 21,
 'reggaeton': 21,
 'brill': 20,
 'dancehall': 20,
 'edm': 19,
 'blues': 19,
 'art': 19,
 'neo': 19,
 'afro': 18,
 'adult': 18,
 'nu': 18,
 'folk': 18,
 'disco': 18,
 'chill': 17,
 'southern': 17,
 'deep': 17,
 'progressive': 16,
 'roots': 16,
 'traditional': 15,
 'south': 13,
 'filmi': 13,
 'australian': 13,
 'electropop': 13,
 'electro': 12,
 'british': 12,
 'europop': 12,
 'emo': 12,
 'acoustic': 11,
 'bebop': 11,
 'big': 11,
 'reggae': 11,
 'metalcore': 11,
 'singer-songwriter': 1

In [11]:
# Create a list of genres that applies to all songs
common_genres = []
song_index_list = []
for index, row in song_artist_category_df.iterrows():
    if row['genres'] == []:
        song_index_list.append(index)

counter = 0
while len(set(song_index_list)) < 926:
    new_genre = list(popular_genres.keys())[counter]
    new_indeces = []
    for index, row in song_artist_category_df.iterrows():
        if (new_genre in row['genres']) & (index not in song_index_list):
            new_indeces.append(index)
    if new_indeces != []:
        song_index_list.extend(new_indeces)
        common_genres.append(new_genre)
    counter+=1
len(common_genres)

110

In [12]:
# Add a column of filtered genres
filtered_genres = []
for artist_genres in song_artist_category_df['genres']:
    filtered = []
    for genre in artist_genres:
        if genre in common_genres:
            filtered.append(genre)
    filtered_genres.append(filtered)
song_artist_category_df['filtered_genres'] = filtered_genres
song_artist_category_df

,song,song_id,artist,artist_id,category,genres,filtered_genres
0,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[post-teen, dance, pop]","[dance, pop]"
1,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[dance, uk, pop]","[dance, uk, pop]"
2,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[electropop, pop]",[pop]
3,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[dance, pop, canadian, post-teen, viral]","[dance, pop, canadian, viral]"
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[queer, lgbtq+, country, pop]","[country, pop]"
...,...,...,...,...,...,...,...
921,Given-Taken,69WpV0U7OMNFGyq8I63dcC,ENHYPEN,5t5FqBwTcgKTaWmfEbwQY9,kpop,[],[]
922,Christmassy!,1KMc5CXwEkjklwSqPMQjB1,THE BOYZ,0CmvFWTX9zmMNCUi6fHtAx,kpop,[k-pop],[k-pop]
923,Better,2k44rKotfi2k55hwwiCImN,BoA,4muJrGMndyYWqZtfk8OWy4,kpop,"[k-pop, j-idol]",[k-pop]
924,Panorama,0CnpSTdL9l5vQM4YnzXtyo,IZ*ONE,5r1tUTxVSgvBHnoDuDODPH,kpop,[k-pop],[k-pop]


In [13]:
# Create dictionary for song features
audio_features = dict.fromkeys(['danceability', 'energy', 'key', 'loudness', 'mode', 
                                'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence', 'tempo', 'duration_ms',
                                'time_signature'], [])
audio_features['danceability'].append('?')
audio_features

{'danceability': ['?'],
 'energy': ['?'],
 'key': ['?'],
 'loudness': ['?'],
 'mode': ['?'],
 'speechiness': ['?'],
 'acousticness': ['?'],
 'instrumentalness': ['?'],
 'liveness': ['?'],
 'valence': ['?'],
 'tempo': ['?'],
 'duration_ms': ['?'],
 'time_signature': ['?']}

In [14]:
# Create lists of audio features
audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                  'duration_ms', 'time_signature']
danceability = [] 
energy = [] 
key = []
loudness = [] 
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []
time_signature = []
for song_id in song_artist_category_df['song_id']:
    query_url = base_url + f'audio-features/{song_id}'
    features = requests.get(query_url, headers={
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {spotify_token}',
        'country': 'US'}).json()
    danceability.append(features['danceability'])
    energy.append(features['energy'])
    key.append(features['key'])
    loudness.append(features['loudness'])
    mode.append(features['mode'])
    speechiness.append(features['speechiness'])
    acousticness.append(features['acousticness'])
    instrumentalness.append(features['instrumentalness'])
    liveness.append(features['liveness'])
    valence.append(features['valence'])
    tempo.append(features['tempo'])
    duration_ms.append(features['duration_ms'])
    time_signature.append(features['time_signature'])
len(danceability)

926

In [15]:
# Add columns of audio features
song_artist_category_df['audio_ft_danceability'] = danceability
song_artist_category_df['audio_ft_energy'] = energy
song_artist_category_df['audio_ft_key'] = key
song_artist_category_df['audio_ft_loudness'] = loudness
song_artist_category_df['audio_ft_mode'] = mode
song_artist_category_df['audio_ft_speechiness'] = speechiness
song_artist_category_df['audio_ft_acousticness'] = acousticness
song_artist_category_df['audio_ft_instrumentalness'] = instrumentalness
song_artist_category_df['audio_ft_liveness'] = liveness
song_artist_category_df['audio_ft_valence'] = valence
song_artist_category_df['audio_ft_tempo'] = tempo
song_artist_category_df['audio_ft_duration_ms'] = duration_ms
song_artist_category_df['audio_ft_time_signature'] = time_signature
song_artist_category_df

,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[post-teen, dance, pop]","[dance, pop]",0.736,0.802,0,-4.759,1,0.0864,0.46800,0.00000,0.0940,0.675,144.005,172325,4
1,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[dance, uk, pop]","[dance, uk, pop]",0.702,0.825,6,-3.787,0,0.0601,0.00883,0.00000,0.0674,0.915,102.977,203064,4
2,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[electropop, pop]",[pop],0.889,0.340,11,-7.773,0,0.0697,0.21800,0.13000,0.0550,0.716,94.009,174321,4
3,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[dance, pop, canadian, post-teen, viral]","[dance, pop, canadian, viral]",0.652,0.383,2,-7.076,0,0.0516,0.06760,0.00000,0.0828,0.549,145.765,178994,4
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[queer, lgbtq+, country, pop]","[country, pop]",0.810,0.511,5,-6.924,0,0.1640,0.12000,0.00000,0.0832,0.837,151.947,154998,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,Given-Taken,69WpV0U7OMNFGyq8I63dcC,ENHYPEN,5t5FqBwTcgKTaWmfEbwQY9,kpop,[],[],0.631,0.649,8,-4.939,0,0.0377,0.17000,0.00000,0.3210,0.459,108.070,183090,4
922,Christmassy!,1KMc5CXwEkjklwSqPMQjB1,THE BOYZ,0CmvFWTX9zmMNCUi6fHtAx,kpop,[k-pop],[k-pop],0.702,0.911,2,-3.521,1,0.0772,0.05590,0.00000,0.0482,0.607,107.940,205533,4
923,Better,2k44rKotfi2k55hwwiCImN,BoA,4muJrGMndyYWqZtfk8OWy4,kpop,"[k-pop, j-idol]",[k-pop],0.587,0.801,5,-4.220,1,0.2290,0.17800,0.00000,0.2140,0.705,92.008,199640,4
924,Panorama,0CnpSTdL9l5vQM4YnzXtyo,IZ*ONE,5r1tUTxVSgvBHnoDuDODPH,kpop,[k-pop],[k-pop],0.726,0.963,9,-1.111,0,0.0916,0.04290,0.00157,0.1280,0.769,126.062,222853,4


In [16]:
# Save DataFrame to CSV
song_artist_category_df.to_csv('../../preliminary_dataframes/song_artist_category3.csv', index=False)

In [29]:
# Create a list of all genres
genre_list = []
for artist_genres in genre_column:
    genre_list.extend(artist_genres)
genre_list = list(set(genre_list))
len(genre_list)

409

In [32]:
genre_list = ['afro' if genre.startswith('afro') else genre for genre in genre_list]
genre_list = ['arab' if genre.startswith('arab') else genre for genre in genre_list]
genre_list = ['bass' if genre.startswith('bass') else genre for genre in genre_list]
genre_list = ['british' if genre.startswith('brit') else genre for genre in genre_list]
genre_list = ['bubble' if genre.startswith('bubble') else genre for genre in genre_list]
genre_list = ['chill' if genre.startswith('chill') else genre for genre in genre_list]
genre_list = ['dance' if genre.startswith('dance') else genre for genre in genre_list]
genre_list = ['electric' if genre.startswith('electr') else genre for genre in genre_list]
genre_list = ['euro' if genre.startswith('euro') else genre for genre in genre_list]
genre_list = ['folk' if genre.startswith('folk') else genre for genre in genre_list]
genre_list = ['indie' if genre.startswith('indie') else genre for genre in genre_list]
genre_list = ['neo-classical' if genre.startswith('neoclassical') else genre for genre in genre_list]
genre_list = ['post' if genre.startswith('post') else genre for genre in genre_list]
genre_list = ['reggae' if genre.startswith('reggae') else genre for genre in genre_list]
genre_list = ['rock' if genre.startswith('rock') else genre for genre in genre_list]
genre_list = ['south' if genre.startswith('south') else genre for genre in genre_list]
genre_list = list(set(genre_list))
print(len(genre_list))

376
